In [1]:
#import dependencies
from tensorflow import keras
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers, optimizers
import numpy as np
from matplotlib import pyplot

In [2]:
# download and split the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print("training data = ", x_train.shape)
print("testing data = ", x_test.shape)

training data =  (50000, 32, 32, 3)
testing data =  (10000, 32, 32, 3)


In [3]:
# Normalize the data to speed up training
mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

# let's look at the normalized values of a sample image
x_train[0]

array([[[-0.9619245 , -0.91515917, -0.8995707 ],
        [-1.2113398 , -1.1645745 , -1.1801629 ],
        [-1.1022207 , -1.1333976 , -1.2113398 ],
        ...,
        [ 0.58133286,  0.17603296, -0.19809005],
        [ 0.48780212,  0.06691375, -0.2916208 ],
        [ 0.4254483 ,  0.05132529, -0.27603233]],

       [[-1.6322283 , -1.5698744 , -1.5698744 ],
        [-1.8816435 , -1.8816435 , -1.8816435 ],
        [-1.6010513 , -1.756936  , -1.8816435 ],
        ...,
        [ 0.03573683, -0.5098592 , -1.0242784 ],
        [-0.026617  , -0.5878015 , -1.1022207 ],
        [ 0.02014837, -0.52544767, -0.9931014 ]],

       [[-1.4919322 , -1.5075206 , -1.554286  ],
        [-1.6322283 , -1.7725244 , -1.8816435 ],
        [-1.117809  , -1.4607552 , -1.756936  ],
        ...,
        [-0.04220546, -0.57221305, -1.1022207 ],
        [-0.01102854, -0.57221305, -1.1022207 ],
        [-0.18250158, -0.7436861 , -1.2269284 ]],

       ...,

       [[ 1.3607558 ,  0.7683944 , -0.38515157],
        [ 1

In [4]:
# one-hot encode the labels in train and test datasets
# we use “to_categorical” function in tensorflow.keras 

num_classes = 10
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

# let's display one of the one-hot encoded labels
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [5]:
# build the model

# number of hidden units variable 
# we are declaring this variable here and use it in our CONV layers to make it easier to update from one place
base_hidden_units = 32

# l2 regularization hyperparameter
weight_decay = 1e-4 

# instantiate an empty sequential model 
model = Sequential()

# CONV1
# notice that we defined the input_shape here because this is the first CONV layer. 
# we don’t need to do that for the remaining layers
model.add(Conv2D(base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV2
model.add(Conv2D(base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# CONV3
model.add(Conv2D(2*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV4
model.add(Conv2D(2*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# CONV5
model.add(Conv2D(4*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# CONV6
model.add(Conv2D(4*base_hidden_units, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# FC7
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [6]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )

# compute the data augmentation on the training set
datagen.fit(x_train)

In [ ]:
# training
batch_size = 64
epochs=125
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint   

checkpointer = ModelCheckpoint(filepath='model.125epochs.hdf5', verbose=1, save_best_only=True)

# you can try any of these optimizers by uncommenting the line
# optimizer = tensorflow.keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
# optimizer = tensorflow.keras.optimizers.adam(lr=0.0005,decay=1e-6)

optimizer = tf.keras.optimizers.RMSprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), callbacks=[checkpointer],
                steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,verbose=2,
                validation_data=(x_test,y_test))


/usr/local/lib64/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/usr/local/lib64/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/125
781/781 - 16s - loss: 2.1423 - accuracy: 0.3692 - val_loss: 1.3907 - val_accuracy: 0.5270

Epoch 00001: val_loss improved from inf to 1.39069, saving model to model.125epochs.hdf5
Epoch 2/125
781/781 - 13s - loss: 1.5310 - accuracy: 0.5019 - val_loss: 1.2992 - val_accuracy: 0.5765

Epoch 00002: val_loss improved from 1.39069 to 1.29919, saving model to model.125epochs.hdf5
Epoch 3/125
781/781 - 13s - loss: 1.3046 - accuracy: 0.5717 - val_loss: 1.0532 - val_accuracy: 0.6558

Epoch 00003: val_loss improved from 1.29919 to 1.05316, saving model to model.125epochs.hdf5
Epoch 4/125
781/781 - 13s - loss: 1.1597 - accuracy: 0.6172 - val_loss: 1.0448 - val_accuracy: 0.6696

Epoch 00004: val_loss improved from 1.05316 to 1.04483, saving model to model.125epochs.hdf5
Epoch 5/125
781/781 - 13s - loss: 1.0524 - accuracy: 0.6526 - val_loss: 0.9590 - val_accuracy: 0.6898

Epoch 00005: val_loss improved from 1.04483 to 0.95899, saving model to model.125epochs.hdf5
Epoch 6/125
781/781 - 13


Epoch 00048: val_loss did not improve from 0.49248
Epoch 49/125
781/781 - 13s - loss: 0.5030 - accuracy: 0.8498 - val_loss: 0.5296 - val_accuracy: 0.8503

Epoch 00049: val_loss did not improve from 0.49248
Epoch 50/125
781/781 - 13s - loss: 0.4986 - accuracy: 0.8536 - val_loss: 0.5019 - val_accuracy: 0.8547

Epoch 00050: val_loss did not improve from 0.49248
Epoch 51/125
781/781 - 13s - loss: 0.4959 - accuracy: 0.8539 - val_loss: 0.5497 - val_accuracy: 0.8478

Epoch 00051: val_loss did not improve from 0.49248
Epoch 52/125
781/781 - 13s - loss: 0.4950 - accuracy: 0.8533 - val_loss: 0.5080 - val_accuracy: 0.8585

Epoch 00052: val_loss did not improve from 0.49248
Epoch 53/125
781/781 - 13s - loss: 0.4918 - accuracy: 0.8545 - val_loss: 0.5346 - val_accuracy: 0.8524

Epoch 00053: val_loss did not improve from 0.49248
Epoch 54/125
781/781 - 13s - loss: 0.4912 - accuracy: 0.8559 - val_loss: 0.5138 - val_accuracy: 0.8552

Epoch 00054: val_loss did not improve from 0.49248
Epoch 55/125
781/7


Epoch 00101: val_loss did not improve from 0.44357
Epoch 102/125
781/781 - 13s - loss: 0.4365 - accuracy: 0.8751 - val_loss: 0.4608 - val_accuracy: 0.8791

Epoch 00102: val_loss did not improve from 0.44357
Epoch 103/125
781/781 - 13s - loss: 0.4299 - accuracy: 0.8785 - val_loss: 0.4887 - val_accuracy: 0.8717

Epoch 00103: val_loss did not improve from 0.44357
Epoch 104/125
781/781 - 13s - loss: 0.4315 - accuracy: 0.8784 - val_loss: 0.4824 - val_accuracy: 0.8731

Epoch 00104: val_loss did not improve from 0.44357
Epoch 105/125
781/781 - 12s - loss: 0.4288 - accuracy: 0.8788 - val_loss: 0.4756 - val_accuracy: 0.8758

Epoch 00105: val_loss did not improve from 0.44357
Epoch 106/125
781/781 - 13s - loss: 0.4340 - accuracy: 0.8779 - val_loss: 0.4561 - val_accuracy: 0.8770

Epoch 00106: val_loss did not improve from 0.44357
Epoch 107/125
781/781 - 13s - loss: 0.4313 - accuracy: 0.8790 - val_loss: 0.4596 - val_accuracy: 0.8779

Epoch 00107: val_loss did not improve from 0.44357
Epoch 108/12

In [ ]:
# evaluating the model
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

In [ ]:
# plot learning curves of model accuracy
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()